# Задача на CTF от компании QRate
**Категории**: криптография, стеганография

**Материалы**: картинка vernam.jpeg (разрешение:800x800, вес: 55МБ)

**Текст задачи**: Вам досталась редкая диковинка: портрет Гильберта Вернама, известного криптографа. Сможете ли вы разгадать его секрет?

## Введение

Так как QRate - это компания, которая занимается квантовой криптографией, мы хотели включить в задачу что-то связанное с нашей работой.

Основная цель криптографии - это безопасная коммуникация, содержание которой остается в секрете. Для этого криптографы используют различные шифры и алгоритмы. Безопасность в них гарантируется сложностью и необычностью математики, используемой для шифрования.

Квантовая криптография в противовес этому фокусируется не на самих шифрах, но на ключах, которые в них используются. Есть ряд довольно простых шифров, гарантирующих абсолютную невозможность взлома (даже с точки зрения теории). Единственное условие - наличие у Алисы и Боба одинакового истинно случайного ключа, который будет использован в процессе. Именно этого мы и добиваемся.

Наиболее распространенный вариант такого шифрования - это шифр Вернама: ключ и сообщение складываются по модулю два сначала Алисой для шифровки, а затем Бобом для расшифровки. При этом, ключ может быть использован для шифрования только одной части сообщения одинаковой с ним длины. Именно нарушение этого правила делает возможной атаку, которую мы предлагаем участникам осуществить в нашей задаче.

Почему же нельзя использовать ключ несколько раз? Дело в том, что многократное повторное использование секретного ключа дает возможность сделать частотный анализ. Злоумышленник (Ева) всё еще не знает, как расшифровать те или иные зашифрованные символы, но он может посчитать их количество в потоке информации, после чего определить язык сообщения. Сопоставив гистограмму частоты символов языка с полученной, Ева может расшифровать часть символов, вычислить с их помощью ключ, и узнать содержание всего сообщения.

Поскольку созданная нами задача довольно сложна, в ней спрятаны дополнительные подсказки, которые могут ускорить и упростить решение.

Карта задачи выглядит следующим образом:

In [ ]:
from ipywidgets import interact
from PIL import Image
import numpy as np

In [13]:
initial_image = Image.open("vernam.jpeg")
initial_image_cropped = initial_image.crop((100, 625, 700, 775))
initial_image_cropped = initial_image_cropped.resize((900,300))

saturated_image = Image.open("vernam_saturated.png")
saturated_image_cropped = saturated_image.crop((100, 625, 700, 775))
saturated_image_cropped = saturated_image_cropped.resize((900,300))

saturated_blured_image = Image.open("vernam_saturated_blured.png")
saturated_blured_image_cropped = saturated_blured_image.crop((100, 625, 700, 775))
saturated_blured_image_cropped = saturated_blured_image_cropped.resize((900,300))

saturated_blured_image_cropped_array = np.array(saturated_blured_image_cropped)
saturated_blured_image_cropped_array_HSV = np.array(saturated_blured_image_cropped.convert("HSV"))

@interact(saturation = (0.0, 1.0), mode = ['normal', 'isolate color'])
def saturate_image(mode, saturation = 0):

    if(mode == 'normal'):
        resulting_image = Image.blend(initial_image_cropped, saturated_image_cropped, saturation)
        display(resulting_image)
        
    if(mode == 'isolate color'):
        lower_mask = saturated_blured_image_cropped_array_HSV [:,:,2] >= 0
        upper_mask = saturated_blured_image_cropped_array_HSV [:,:,2] <= 231
        mask = upper_mask * lower_mask
        red = saturated_blured_image_cropped_array[:,:,0]*mask
        green = saturated_blured_image_cropped_array[:,:,1]*mask
        blue = saturated_blured_image_cropped_array[:,:,2]*mask
        green_mask = Image.fromarray(np.dstack((red,green,blue)))
        resulting_image_green = Image.blend(initial_image_cropped, green_mask, saturation/1.3)
        display(resulting_image_green)

interactive(children=(Dropdown(description='mode', options=('normal', 'isolate color'), value='normal'), Float…

In [128]:
key = 'mctf{try_using_QRNG}'

def string_to_binary_utf8(key):
    return ''.join('0' + format(ord(x), 'b') for x in key)

def binary_utf8_to_string(message):
    return ''.join(chr(int(x, 2)) for x in [message[i:i + 8] for i in range(0, len(message), 8)])

def string_to_binary_pixels(key):
    numbers = [['0110', '1001', '1001', '1001', '0110'], ['0010', '0110', '0010', '0010', '0111']]
    return ''.join(list((numbers[key[i%8] == '1'][i//8] for i in range (40))))

print(binary_utf8_to_string(string_to_binary_utf8(key)))
str = string_to_binary_utf8('a')
print(len(string_to_binary_pixels(str)))

mctf{try_using_QRNG}
160
